Continue with v1, but dynamically update the parameters weekly (5 trading days per week)

In [1]:
from rectangle_fuzzy import *

import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt
%matplotlib inline
from time import time

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# helper functions
def binary_to_decimal(binary, di=0):
    decimal = 0
    for i in range(len(binary)):
        if i<len(binary)-di+1:
            power = len(binary)-(i+1)-di
            decimal += int(binary[i]) * (2**power)
        else:
            power = len(binary)-(i+1)-di
            decimal += int(binary[i]) * (2**(-power))
    return decimal

def universe_partition_adjusted(data, n=9, d1=9, d2=10):
    x_max, x_min = data.max(axis=0), data.min(axis=0)   
    delta = x_max - x_min
    std_val = data.std(axis=0)
    len_val = np.round(std_val / (n+5)) # partition interval
    u_max, u_min = int(x_max+(d2+1)*(delta/100)), int(x_min-(d1+1)*(delta/100)) # bound of universe discourse
    u_b = np.arange(u_min, u_max, step=float(len_val)) # cutting points
    u_discourse = u_b
    return u_discourse

def compute_fitness(data, n, d1, d2, option='Price'):
    reserved_days = 5
    first_date = data['Date'][0] - pd.DateOffset(days=1)
    train_data = data.iloc[:-reserved_days]
    valid_data = data.iloc[-(reserved_days+1):]
    fit_prices = list()
#     u_discourse = universe_partition_adjusted(train_data[option], n, d1, d2)
    u_discourse = universe_partition_adjusted(train_data[option], n)
    A_list = set_fuzzy_numbers(u_discourse)
    train_data_membership_series = membership_assignment(train_data[option], fuzzy_numbers=u_discourse)
    transition_FLR = FLR(train_data_membership_series)
    train_data_days = train_data['Date'].apply(lambda x: x-first_date).dt.days.tolist()
    jump_weights = FLR_weight(transition_FLR, train_data_days[:len(transition_FLR)])
    transition_weights = FRG_weight(transition_FLR, train_data_days[:len(transition_FLR)])
    for price in valid_data.iloc[:-1][option]:
        index = get_membership(price, u_discourse)
        next_price = forecast_price(forecast_jump(index, jump_weights, A_list), forecast_transition(index, transition_weights, A_list), gamma=0.9)
        fit_prices.append(next_price)
    return np.sqrt(mean_squared_error(valid_data.iloc[1:][option], fit_prices))

def parameter_tuning_ASHLO(data, option):
    # initialize hyperparameters
    d1_digits = 0
    n_digits = 6
    d2_digits = 0
    population = 20
    iter_max = 100
    pr = 0.1
    pi = 0.9
    # initialize data structures
    dim_IKD = d1_digits+n_digits+d2_digits
    RMSE_list = list() # training data RMSE as fitness function value
    IKD = np.zeros((population, dim_IKD), dtype=int) # individual learning 
    SKD = np.array([]) # social network learning
    RMSE_min = 0 # best fitness value
    for i in range(population):
        for j in range(dim_IKD):
            r = np.random.rand()
            if r>0.5:
                IKD[i, j] = 0
            else:
                IKD[i, j] = 1
        d1 = binary_to_decimal(IKD[i, 0:d1_digits])
        d2 = binary_to_decimal(IKD[i, (d1_digits+n_digits): dim_IKD])
        n_par = binary_to_decimal(IKD[i, d1_digits:(d1_digits+n_digits)])
        rmse = compute_fitness(data, n_par, d1, d2, option)
        RMSE_list.append(rmse)
    RMSE_min = np.min(RMSE_list) # best individual fitness
    SKD = np.array(IKD[np.argmin(RMSE_list)])
    # learning iterations
    for iter_count in range(iter_max):
        temp_IKD = np.zeros((population, dim_IKD))
        temp_rmse_list = list()
        # learning process
        for i in range(population):
            for j in range(dim_IKD):
                p = np.random.rand()
                if p < pi:
                    r = np.random.rand()
                    if r > 0.5:
                        temp_IKD[i, j] = 1
                elif pi <= p <= pr:
                    temp_IKD[i,j] = IKD[i, j]
                else:
                    temp_IKD[i,j] = SKD[j]
        # update fitness of newly learned knowledge 
        for i in range(population):
            d1 = binary_to_decimal(temp_IKD[i, 0:d1_digits])
            d2 = binary_to_decimal(temp_IKD[i, (d1_digits+n_digits): dim_IKD])
            n_par = binary_to_decimal(temp_IKD[i, d1_digits:(d1_digits+n_digits)])
            rmse = compute_fitness(data, n_par, d1, d2, option)
            temp_rmse_list.append(rmse)

        # update knowledge base if necessary
        RMSE_max = np.max(RMSE_list) # worst individual fitness
        RMSE_max_index = np.argmax(RMSE_list)
        for i in range(population):
            if temp_rmse_list[i] < RMSE_max:
                IKD[RMSE_max_index] = temp_IKD[i]
                RMSE_list[RMSE_max_index] = temp_rmse_list[i]
                RMSE_max = np.max(RMSE_list)
                RMSE_max_index = np.argmax(RMSE_list)
        temp_rmse_min = np.min(RMSE_list) # best individual fitness
        if RMSE_min > temp_rmse_min:
            SKD = np.array(IKD[np.argmin(RMSE_list)])
            RMSE_min = temp_rmse_min
    # output optimal parameter settings        
    d1 = binary_to_decimal(SKD[0:d1_digits])
    d2 = binary_to_decimal(SKD[(d1_digits+n_digits): ])
    n_par = binary_to_decimal(SKD[d1_digits:(d1_digits+n_digits)])
    return d1, n_par, d2

In [3]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100 # beware of zero division

In [4]:
# Load data 
start_time = time()
datafile = '../data/NI225.csv'
dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
df = pd.read_csv(datafile, header=0, parse_dates=['Date'], date_parser=dateparse)

split_date = pd.datetime.strptime('2014-12-31', '%Y-%m-%d')
train_data = df[df['Date'] < split_date]
test_data = df[df['Date'] >= split_date]
first_date = train_data['Date'][0] - pd.DateOffset(days=1)

option = 'Open'
update_interval = 5 # days

In [5]:
# forecasting using test data
start_time = time()
pred_prices = list()
error_percentages = list() # percentage of forecasting residual
actual_prices = test_data[option].tolist()[1:]
indices = test_data.index
for i in range(len(indices)-1):
    print("Iteration #%s" % str(i))
    prev = test_data.iloc[i]
    train_data.append(prev)
    if (i % update_interval)==0:
        print("\tUpdate parameter setting and fuzzy sets ...")
        d1, n_par, d2 = parameter_tuning_ASHLO(train_data, option)
        print("\t\td1=%f, d2=%f, n=%f" % (d1, d2, n_par))
#         u_discourse = universe_partition_adjusted(train_data[option], n_par, d1, d2)
        u_discourse = universe_partition_adjusted(train_data[option], n_par)
        A_list = set_fuzzy_numbers(u_discourse)
        train_data_membership_series = membership_assignment(train_data[option], fuzzy_numbers=u_discourse)
        transition_FLR = FLR(train_data_membership_series)
        train_data_days = train_data['Date'].apply(lambda x: x-first_date).dt.days.tolist() # convert Timedelta to numeric days
        jump_weights = FLR_weight(transition_FLR, train_data_days[:len(transition_FLR)])
        transition_weights = FRG_weight(transition_FLR, train_data_days[:len(transition_FLR)])
    price = prev[option]
    index = get_membership(price, u_discourse)
    next_price = forecast_price(forecast_jump(index, jump_weights, A_list), forecast_transition(index, transition_weights, A_list), gamma=0.9)
    pred_prices.append(next_price)
    pred_ep = (np.abs(actual_prices[i] - next_price)/actual_prices[i])
    error_percentages.append(pred_ep)
    print("\t---- error percentage = %f" % pred_ep)
    
print "MAD = "+str(mean_absolute_error(actual_prices, pred_prices))
print "RMSE = "+str(np.sqrt(mean_squared_error(actual_prices, pred_prices)))
print "MAPE = "+str(mean_absolute_percentage_error(actual_prices, pred_prices))
print "Time cost: "+str(time()-start_time)+" seconds" 

Iteration #0
	Update parameter setting and fuzzy sets ...
		d1=0.000000, d2=0.000000, n=12.000000
	---- error percentage = 0.014296
Iteration #1
	---- error percentage = 0.015839
Iteration #2
	---- error percentage = 0.014857
Iteration #3
	---- error percentage = 0.014103
Iteration #4
	---- error percentage = 0.022108
Iteration #5
	Update parameter setting and fuzzy sets ...
		d1=0.000000, d2=0.000000, n=12.000000
	---- error percentage = 0.000664
Iteration #6
	---- error percentage = 0.005934
Iteration #7
	---- error percentage = 0.005153
Iteration #8
	---- error percentage = 0.010997
Iteration #9
	---- error percentage = 0.005774
Iteration #10
	Update parameter setting and fuzzy sets ...
		d1=0.000000, d2=0.000000, n=12.000000
	---- error percentage = 0.013533
Iteration #11
	---- error percentage = 0.002278
Iteration #12
	---- error percentage = 0.009963
Iteration #13
	---- error percentage = 0.014249
Iteration #14
	---- error percentage = 0.021813
Iteration #15
	Update parameter set

KeyboardInterrupt: 

In [ ]:
df_test = pd.DataFrame(test_data.iloc[1:]['Date'])
df_test['data testing'] = pd.Series(actual_prices).values
df_test['forecasting'] = pd.Series(pred_prices).values
df_test.index = df_test['Date']
plt.figure(figsize=(14,7))
df_test['data testing'].plot(style='k-', linewidth=2, label='data testing')
df_test['forecasting'].plot(style='g:', linewidth=3, label='forecasting')
plt.ylabel('Price', fontsize=18)
plt.legend(loc='best')
plt.title("NI225_forecasting", fontsize=24)

In [ ]:
plt.figure(figsize=(14,7))
df_test["residual"] = df_test["data testing"] - df_test["forecasting"]
df_test["residual"].plot(style='bo', linewidth=3, label='residual')
plt.ylabel('Residual', fontsize=18)
plt.axhline(y=0.0, color='k', linestyle='--')
plt.legend(loc='best')
plt.title("NI225_forecasting_residual", fontsize=24)